## Data Information


Version 1.0
March 2010

. What is this?

    This dataset contains <user, artist, song, timestamp> tuples collected from Last.fm API ( http://www.last.fm/api ), 
    using the user.getRecentTracks method ( http://www.last.fm/api/show?service=278 )

    This dataset represents the whole listening habits (till May, 5th 2009) for nearly 1,000 users.

. Data Format:

    The data is formatted one entry per line as follows (tab separated, "\t"):

    userid-timestamp-artid-artname-traid-traname.tsv
      userid \t timestamp \t musicbrainz-artist-id \t artist-name \t musicbrainz-track-id \t track-name

    userid-profile.tsv:
      userid \t gender ('m'|'f'|empty) \t age (int|empty) \t country (str|empty) \t signup (date|empty)

. Example:

    userid-timestamp-artid-artname-traid-traname.tsv:
      user_000639  2009-04-08T01:57:47Z  15676fc4-ba0b-4871-ac8d-ef058895b075  The Dogs D'Amour  6cc252d0-3f42-4fd3-a70f-c8ff8b693aa4  How Do You Fall in Love Again
      user_000639  2009-04-08T01:53:56Z  15676fc4-ba0b-4871-ac8d-ef058895b075  The Dogs D'Amour  aa7dbea2-a0c0-4d0a-9241-5bb98a372b11  Wait Until I'm Dead
      ...

    userid-profile.tsv:
      user_000639     m        Mexico    Apr 27, 2005
      ...

. Data Statistics:

     Total Lines:           19,150,868
     Unique Users:                 992
     Artists with MBID:        107,528
     Artists without MBDID:     69,420

. Files:

    userid-timestamp-artid-artname-traid-traname.tsv (MD5: 64747b21563e3d2aa95751e0ddc46b68)
    userid-profile.tsv                               (MD5: c53608b6b445db201098c1489ea497df)

. License:

    The data in lastfm-dataset-1K is distributed with permission of Last.fm. 
    The data is made available for non-commercial use.
    Those interested in using the data or web services in a commercial context 
    should contact: partners [at] last [dot] fm. 
    For more information see http://www.last.fm/api/tos

. Acknowledgements:

    Thanks to Last.fm for providing the access to this data via their web services. 
    Special thanks to Norman Casagrande.

. Contact:

    This data was collected by Oscar Celma. Send questions or comments to oscar.celma@upf.edu

#Task
====

You are the a Data Engineer at First.AM and the product team (Bob) need your help with a data set that does not fit into Excel. He wants some basic analytics generated as a regular report. Your CTO asked that all new data engineering projects need to be written professionally and should use Spark. The CTO does allow the use of Python, Scala and Java.

Q: Bob would like to know how many tracks were listened to in total.

Q: Bob would like to know how many unique users we had. 

Q: Bob would like to know what the most popular track was for women vs men in July 2008.

Q: Bob would like a clean, complete, denormalised data set to load into an RDBMS, i.e. join unique and complete rows from both files to one large output.


#Data
====

The data-information file has some information about the actual source data, note that the data has been slightly changed for this scenario.

Use only the files preceeding with "t-" for development steps. Use the "r-" files to simulate production runs.

#Setup
=====
You can use Python, Scala, or Java and should use Spark if possible. You can use the internet to look up information and use UIs as you would at work. You should treat the task something that is going into production and not merely as a quick and dirty script. Your interviewer can answer any questions you might have for Bob.

#Code Implementation

#Loading track records and recreating a temp table

In [0]:
# File location and type
track_records_location = "/FileStore/tables/track_records/userid-timestamp-artid-artname-traid-traname.tsv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "false"
delimiter = "\t"

# The applied options are for CSV files. For other file types, these will be ignored.
recent_tracks_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(track_records_location) \
  .toDF("userid","timestamp","musicbrainz-artist-id","artist-name","musicbrainz-track-id","track-name")

recent_tracks_df.cache()
recent_tracks_df.createOrReplaceTempView("recent_tracks")

recent_tracks_df.show()


#Loading user profile and recreating a temp table

In [0]:
userid_profile_location = "/FileStore/tables/profile/r_userid_profile.tsv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = "\t"

# The applied options are for CSV files. For other file types, these will be ignored.
profile_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(userid_profile_location) \

#Rename the columns to required/proper names 
profile_df = profile_df.withColumnRenamed("#id", "userid") \
.withColumnRenamed("registered","signup")

profile_df.cache()

profile_df.createOrReplaceTempView("profile")

display(profile_df)

#  Data Statistics on Recent Track Redords

Total Lines

Unique Users

In [0]:
total_tracks_df.show()

In [0]:
recent_tracks_df.dropDuplicates(['userid']).count()

Artists with MBID

In [0]:
spark.sql("select * from uinque_id where -id` is not null").count()

In [0]:
spark.sql("select * from uinque_tracks where -id` is not null").count()

In [0]:
spark.sql("select * from recent_tracks where `musicbrainz-artist-id` is not null").count() #18548705

Artists without MBID

In [0]:
spark.sql("select * from recent_tracks where `musicbrainz-artist-id` is null").count() #602166

#Questions

#Q1 : Bob would like to know how many tracks were listened to in total.

In [0]:
%sql 

select count(*) from recent_tracks;


#Q2: Bob would like to know how many unique users we had.

In [0]:
%sql 

select count(distinct(userid)) as count_unique_users from recent_tracks where userid is not null;



#Q3: Bob would like to know what the most popular track was for women vs men in July 2008.

In [0]:
%sql 

select CASE res_tbl.gender WHEN "m" THEN "MALE" ELSE "FEMALE" END, res_tbl.`track-name`,  res_tbl.max as popular_track_count from (
  SELECT t.gender, t.`track-name`, t.count as count, MAX(t.count) OVER (PARTITION BY t.gender ORDER BY t.gender) AS max
  FROM (
    select gender, `track-name`, count(*) as count from (
      select * from recent_tracks where date_format(timestamp, "yyyy-MM") = '2006-04' and `track-name` is not null) rt 
      join (select * from profile where gender is not null) p
      on rt.userid = p.userid group by gender, rt.`track-name`) t) res_tbl
where res_tbl.max = res_tbl.count;

CASE WHEN (gender = m) THEN MALE ELSE FEMALE END,track-name,popular_track_count
MALE,Vicarious,236
FEMALE,The Promised Land,419


#Q4 : Bob would like a clean, complete, denormalised data set to load into an RDBMS, i.e. join unique and complete rows from both files to one large output.

In [0]:
%sql 
-- checking count records which are hainvg artist-name is snull
select count(*) from recent_tracks where `artist-name` is null;

In [0]:
%sql 
-- checking count records which are hainvg musicbrainz-track-id is snull
select count(*) from recent_tracks where `musicbrainz-track-id` is null;

In [0]:

select count(distinct(userid)) as count_unique_users from recent_tracks where userid is not nul

In [0]:
select count(*) from recent_tracks where `track-name` is null,df_unique_trac.show(),select count(*) from track_id= where trac_id=

In [0]:
select count(*) from recent_tracks where 

In [0]:
%sql 
-- checking count records which are hainvg track-name i snull
select count(*) from recent_tracks where `track-name` is null;

Schema:

Track records :

      userid \t timestamp \t musicbrainz-artist-id \t artist-name \t musicbrainz-track-id \t track-name

I checked for each field as each filed is mandatory and should not be null


profiles:

      userid \t gender ('m'|'f'|empty) \t age (int|empty) \t country (str|empty) \t signup (date|empty)
      
Just cleaned the records which are null in userid, remaining field may contains empty/null values

In [0]:
%sql 
-- Cleaing all the records which are null
--Then joinig two datesets to store in RDBMS 16945473

select count(*) from (
select p.*, r.timestamp, r.`musicbrainz-artist-id`, r.`artist-name`, r.`musicbrainz-track-id`, r.`track-name` from (
  select *  from recent_tracks rt 
  where rt.userid is not null and rt.timestamp is not null and rt.`musicbrainz-artist-id` is not null and  rt.`artist-name` is not null and rt.`musicbrainz-track-id` is not null and  rt.`track-name` is not null) r
left outer join (
  select * from profile where userid is not null) p
on r.userid = p.userid
where p.userid is not null)